<a href="https://colab.research.google.com/github/rashida048/hypermeter-tuning/blob/main/hyperparamTuning_deep_learning_inScikitLearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com/hyperparameter-tuning-tf-scikit/hyperparameter-tuning-tf-scikit.zip
!unzip -qq hyperparameter-tuning-tf-scikit.zip
%cd hyperparameter-tuning-tf-scikit

--2023-06-05 20:33:03--  https://pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com/hyperparameter-tuning-tf-scikit/hyperparameter-tuning-tf-scikit.zip
Resolving pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com (pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com)... 3.5.76.103, 52.218.233.129, 52.218.183.42, ...
Connecting to pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com (pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com)|3.5.76.103|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4941 (4.8K) [binary/octet-stream]
Saving to: ‘hyperparameter-tuning-tf-scikit.zip’

hyperparameter-tuni 100%[===================>]   4.83K  --.-KB/s    in 0s      

2023-06-05 20:33:04 (169 MB/s) - ‘hyperparameter-tuning-tf-scikit.zip’ saved [4941/4941]

/content/hyperparameter-tuning-tf-scikit


In [2]:
# import tensorflow and fix the random seed for better reproducibility
import tensorflow as tf
tf.random.set_seed(42)

# import the necessary packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.datasets import mnist

In [3]:
def get_mlp_model(hiddenLayerOne=784, hiddenLayerTwo=256,
	dropout=0.2, learnRate=0.01):
	# initialize a sequential model and add layer to flatten the
	# input data
	model = Sequential()
	model.add(Flatten())

	# add two stacks of FC => RELU => DROPOUT
	model.add(Dense(hiddenLayerOne, activation="relu",
		input_shape=(784,)))
	model.add(Dropout(dropout))
	model.add(Dense(hiddenLayerTwo, activation="relu"))
	model.add(Dropout(dropout))

	# add a softmax layer on top
	model.add(Dense(10, activation="softmax"))

	# compile the model
	model.compile(
		optimizer=Adam(learning_rate=learnRate),
		loss="sparse_categorical_crossentropy",
		metrics=["accuracy"])

	# return compiled model
	return model

In [4]:
# load the MNIST dataset
print("[INFO] downloading MNIST...")
((trainData, trainLabels), (testData, testLabels)) = mnist.load_data()

# scale data to the range of [0, 1]
trainData = trainData.astype("float32") / 255.0
testData = testData.astype("float32") / 255.0

[INFO] downloading MNIST...
11490434/11490434 [==============================] - 1s 0us/step


In [5]:
model = KerasClassifier(build_fn=get_mlp_model, verbose=0)

# define a grid of the hyperparameter search space
hiddenLayerOne = [256]
hiddenLayerTwo = [128, 256]
learnRate = [1e-2, 1e-3]
dropout = [0.4]
batchSize = [32]
epochs = [40]

# create a dictionary from the hyperparameter grid
grid = dict(
	hiddenLayerOne=hiddenLayerOne,
	learnRate=learnRate,
	hiddenLayerTwo=hiddenLayerTwo,
	dropout=dropout,
	batch_size=batchSize,
	epochs=epochs
)

<ipython-input-5-588794ea7873>:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=get_mlp_model, verbose=0)


In [8]:
searcher = RandomizedSearchCV(estimator=model, n_jobs=1, 
                              cv = 3, param_distributions=grid, scoring='accuracy')
searchResults = searcher.fit(trainData, trainLabels)

625/625 [==============================] - 1s 2ms/step


In [9]:
bestScore = searchResults.best_score_
bestParams = searchResults.best_params_ 
print("[INFO] best score is {:.2f} using {}".format(bestScore,
	bestParams))

[INFO] best score is 0.98 using {'learnRate': 0.001, 'hiddenLayerTwo': 128, 'hiddenLayerOne': 256, 'epochs': 40, 'dropout': 0.4, 'batch_size': 32}


In [10]:
bestModel = searchResults.best_estimator_
accuracy = bestModel.score(testData, testLabels)
print("accuracy: {:.2f}%".format(accuracy * 100))

accuracy: 98.26%
